# Biểu diễn Doc2vec sử dụng Deep Learning

## Có sử dụng pretrained models

Notebook này triển khai mô hình Doc2vec sử dụng Deep Learning với pretrained model Word2Vec. Cụ thể:

1. Sử dụng pretrained Word2Vec (GloVe) làm khởi tạo cho word embeddings
2. Xây dựng mô hình Deep Neural Network với nhiều lớp ẩn để tạo document embeddings
3. Kết hợp cả hai để biểu diễn văn bản dưới dạng vector

In [ ]:
# Import các thư viện cần thiết
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
from gensim.downloader import load
from collections import defaultdict
import logging
import os
import time

# Thiết lập logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
# Tải NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

## 1. Tạo dữ liệu mẫu

Tạo tập dữ liệu văn bản mẫu để thực nghiệm với Doc2vec.

In [ ]:
# Tạo dữ liệu mẫu
def create_sample_data(filename='sample_data.csv'):
    """
    Tạo tập dữ liệu mẫu và lưu thành CSV
    """
    sample_texts = [
        "Deep learning enables machine learning models to learn hierarchical representations.",
        "Neural networks are the foundation of most deep learning methods.",
        "Word embeddings capture semantic relationships between words in a text corpus.",
        "Document embeddings represent entire documents in a vector space.",
        "Natural language processing uses computational techniques to analyze text.",
        "Transfer learning leverages knowledge from pretrained models for new tasks.",
        "Convolutional neural networks excel at computer vision tasks.",
        "Recurrent neural networks are effective for sequential data like text.",
        "Attention mechanisms help models focus on relevant parts of the input data.",
        "Transformers have revolutionized natural language understanding tasks.",
        "Doc2Vec extends Word2Vec to create document-level embeddings.",
        "Unsupervised learning discovers patterns without labeled training data.",
        "Semantic analysis aims to understand the meaning of text data.",
        "Python is the most popular programming language for deep learning.",
        "PyTorch provides a flexible framework for building neural networks."
    ]
    
    # Tạo labels cho task phân loại (để demo)
    labels = ['technique', 'architecture', 'embedding', 'embedding', 'technique',
              'technique', 'architecture', 'architecture', 'component', 'architecture',
              'embedding', 'technique', 'technique', 'tool', 'tool']
    
    df = pd.DataFrame({
        'text': sample_texts,
        'label': labels
    })
    
    # Lưu thành file CSV
    df.to_csv(filename, index=False)
    print(f"Đã tạo file {filename} với {len(df)} văn bản")
    return df

# Tạo dữ liệu mẫu nếu chưa có
if not os.path.exists('sample_data.csv'):
    df = create_sample_data()
else:
    df = pd.read_csv('sample_data.csv')
    print(f"Đã đọc file sample_data.csv với {len(df)} văn bản")
    
# Hiển thị 5 mẫu đầu tiên
df.head()

## 2. Tải Pretrained Word Embeddings

Tải pretrained word embeddings từ Gensim. Chúng ta sử dụng GloVe với 100 chiều để giảm thời gian tải và tiết kiệm bộ nhớ.

In [ ]:
def download_word_embeddings(model_name='glove-wiki-gigaword-100'):
    """
    Tải pretrained word embeddings và lưu ở định dạng memory-mapped
    """
    pretrained_path = f"{model_name.replace('-', '_')}.bin"
    mmap_path = f"{model_name.replace('-', '_')}.mmap"
    
    if not os.path.exists(mmap_path):
        if not os.path.exists(pretrained_path):
            print(f"Đang tải model {model_name}...")
            start_time = time.time()
            word_vectors = load(model_name)
            print(f"Đã tải xong sau {time.time() - start_time:.2f} giây")
            
            print("Đang lưu model dưới dạng binary...")
            word_vectors.save_word2vec_format(pretrained_path, binary=True)
            print(f"Đã lưu model tại: {pretrained_path}")
        
        # Chuyển đổi sang định dạng memory-mapped để tối ưu RAM
        print("Đang chuyển sang định dạng memory-mapped...")
        temp_wv = KeyedVectors.load_word2vec_format(pretrained_path, binary=True)
        temp_wv.save(mmap_path)
        vector_size = temp_wv.vector_size
        vocab_size = len(temp_wv)
        del temp_wv  # Giải phóng bộ nhớ
        print(f"Đã lưu memory-mapped model tại: {mmap_path}")
        print(f"Kích thước từ điển: {vocab_size}, Kích thước vector: {vector_size}")
    else:
        print(f"Đang tải memory-mapped model từ: {mmap_path}")
        temp_wv = KeyedVectors.load(mmap_path, mmap='r')
        vector_size = temp_wv.vector_size
        vocab_size = len(temp_wv)
        del temp_wv  # Giải phóng bộ nhớ
        print(f"Kích thước từ điển: {vocab_size}, Kích thước vector: {vector_size}")
    
    return mmap_path, vector_size

# Tải pretrained word embeddings
pretrained_path, VECTOR_SIZE = download_word_embeddings()

## 3. Xây dựng Dataset và Preprocessing

Xây dựng các class để xử lý dữ liệu văn bản và chuẩn bị cho việc huấn luyện.

In [ ]:
class TextDataset:
    def __init__(self, texts, max_length=50):
        """
        Khởi tạo dataset từ danh sách các văn bản
        texts: list các văn bản
        max_length: độ dài tối đa của mỗi văn bản
        """
        self.texts = texts
        self.max_length = max_length
        self.word2idx = {'<PAD>': 0, '<UNK>': 1}  # Thêm token đặc biệt
        self.idx2word = {0: '<PAD>', 1: '<UNK>'}
        self.vocab_size = 2
        
        # Xây dựng từ điển
        self._build_vocab()
        
    def _build_vocab(self):
        """
        Xây dựng từ điển từ tất cả các văn bản
        """
        word_freq = defaultdict(int)
        
        # Đếm tần suất từ
        for text in self.texts:
            words = word_tokenize(text.lower())
            for word in words:
                word_freq[word] += 1
        
        # Tạo từ điển với các từ xuất hiện ít nhất 1 lần
        for word, freq in word_freq.items():
            if freq >= 1 and word not in self.word2idx:
                self.word2idx[word] = self.vocab_size
                self.idx2word[self.vocab_size] = word
                self.vocab_size += 1
        
        print(f"Kích thước từ điển: {self.vocab_size}")
        
    def tokenize_text(self, text):
        """
        Chuyển văn bản thành chuỗi các index
        """
        words = word_tokenize(text.lower())
        return [self.word2idx.get(word, self.word2idx['<UNK>']) for word in words][:self.max_length]

# Chia dữ liệu thành train và validation
train_texts, val_texts = train_test_split(df['text'].tolist(), test_size=0.2, random_state=42)

# Tạo dataset cho các văn bản
text_dataset = TextDataset(train_texts + val_texts)

In [ ]:
class Doc2VecDataset(Dataset):
    def __init__(self, texts, text_dataset, window_size=5):
        """
        Dataset cho Doc2Vec
        texts: danh sách các văn bản
        text_dataset: TextDataset object chứa từ điển
        window_size: kích thước cửa sổ ngữ cảnh
        """
        self.texts = texts
        self.text_dataset = text_dataset
        self.window_size = window_size
        self.data = self._prepare_training_data()
        
    def _prepare_training_data(self):
        """
        Chuẩn bị dữ liệu huấn luyện với cửa sổ ngữ cảnh
        Format: (doc_id, target_word_id, context_word_id)
        """
        training_data = []
        
        for doc_id, text in enumerate(self.texts):
            word_indices = self.text_dataset.tokenize_text(text)
            
            for target_pos, target_idx in enumerate(word_indices):
                # Lấy các từ trong cửa sổ ngữ cảnh
                context_indices = []
                start = max(0, target_pos - self.window_size)
                end = min(len(word_indices), target_pos + self.window_size + 1)
                
                for i in range(start, end):
                    if i != target_pos:  # Bỏ qua target word
                        context_indices.append(word_indices[i])
                
                for context_idx in context_indices:
                    training_data.append((doc_id, target_idx, context_idx))
                    
        return training_data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        doc_id, target_idx, context_idx = self.data[idx]
        return (
            torch.tensor(doc_id, dtype=torch.long),
            torch.tensor(target_idx, dtype=torch.long),
            torch.tensor(context_idx, dtype=torch.long)
        )

# Tạo dataset cho train và validation
train_dataset = Doc2VecDataset(train_texts, text_dataset)
val_dataset = Doc2VecDataset(val_texts, text_dataset)

print(f"Số lượng mẫu huấn luyện: {len(train_dataset)}")
print(f"Số lượng mẫu validation: {len(val_dataset)}")

## 4. Xây dựng Mô Hình Doc2Vec với Deep Learning

Xây dựng mô hình Doc2Vec sử dụng Deep Learning với pretrained word embeddings.

In [ ]:
class Doc2VecWithPretrained(nn.Module):
    def __init__(self, pretrained_path, vocab_size, doc_count, vector_size=100):
        """
        Mô hình Doc2Vec với pretrained embeddings và deep learning layers
        pretrained_path: đường dẫn đến pretrained model
        vocab_size: kích thước từ điển
        doc_count: số lượng văn bản
        vector_size: kích thước vector
        """
        super(Doc2VecWithPretrained, self).__init__()
        
        self.vector_size = vector_size
        self.vocab_size = vocab_size
        
        # Word embeddings
        self.word_embeddings = nn.Embedding(vocab_size, vector_size)
        
        # Document embeddings
        self.doc_embeddings = nn.Embedding(doc_count, vector_size)
        
        # Khởi tạo document embeddings
        nn.init.uniform_(self.doc_embeddings.weight, -0.1, 0.1)
        
        # Deep learning layers - Kiến trúc sâu với nhiều lớp ẩn
        self.fc1 = nn.Linear(vector_size * 2, vector_size * 2)
        self.fc2 = nn.Linear(vector_size * 2, vector_size)
        self.fc3 = nn.Linear(vector_size, vector_size)
        self.output = nn.Linear(vector_size, vocab_size)
        
        # Các layer khác
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.3)
        self.activation = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(vector_size * 2)
        self.batch_norm2 = nn.BatchNorm1d(vector_size)
        
        # Khởi tạo word embeddings từ pretrained model
        self._init_word_embeddings(pretrained_path)
        
    def _init_word_embeddings(self, pretrained_path):
        """
        Khởi tạo word embeddings từ pretrained model
        """
        try:
            # Khởi tạo ngẫu nhiên trước
            nn.init.uniform_(self.word_embeddings.weight, -0.1, 0.1)
            
            # Tải pretrained model
            pretrained_vecs = KeyedVectors.load(pretrained_path, mmap='r')
            
            # Map từ word2idx tới pretrained vectors
            initialized = 0
            for word, idx in self.word2idx.items():
                if word in pretrained_vecs:
                    self.word_embeddings.weight.data[idx] = torch.FloatTensor(pretrained_vecs[word])
                    initialized += 1
                elif word.lower() in pretrained_vecs:  # Thử lowercase
                    self.word_embeddings.weight.data[idx] = torch.FloatTensor(pretrained_vecs[word.lower()])
                    initialized += 1
            
            print(f"Đã khởi tạo {initialized}/{self.vocab_size} từ từ pretrained vectors")
        except Exception as e:
            print(f"Lỗi khi khởi tạo word embeddings: {e}")

    def forward(self, doc_ids, word_ids):
        """
        Forward pass
        doc_ids: tensor chứa document IDs
        word_ids: tensor chứa word IDs
        """
        # Lấy embeddings
        doc_embeds = self.doc_embeddings(doc_ids)
        word_embeds = self.word_embeddings(word_ids)
        
        # Kết hợp document và word embeddings
        combined = torch.cat([doc_embeds, word_embeds], dim=1)
        
        # Deep learning layers
        x = self.activation(self.fc1(combined))
        x = self.dropout1(x)
        x = self.batch_norm1(x)
        
        x = self.activation(self.fc2(x))
        x = self.dropout2(x)
        x = self.batch_norm2(x)
        
        x = self.activation(self.fc3(x))
        x = self.dropout3(x)
        
        # Output layer
        output = self.output(x)
        
        return output

## 5. Huấn luyện mô hình

Thiết lập quá trình huấn luyện và validation.

In [ ]:
def train_doc2vec(model, train_loader, val_loader, epochs=5, learning_rate=0.001):
    """
    Huấn luyện mô hình Doc2Vec
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Sử dụng device: {device}")
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    
    # Optimizer với learning rates khác nhau cho word embeddings và doc embeddings
    word_embed_params = {'params': model.word_embeddings.parameters(), 'lr': learning_rate * 0.1}
    doc_embed_params = {'params': model.doc_embeddings.parameters(), 'lr': learning_rate}
    other_params = {'params': [p for n, p in model.named_parameters() 
                             if 'word_embeddings' not in n and 'doc_embeddings' not in n],
                  'lr': learning_rate, 'weight_decay': 1e-5}
    
    optimizer = optim.Adam([word_embed_params, doc_embed_params, other_params])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=1, verbose=True)
    
    # Để theo dõi tiến trình
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    # Early stopping
    patience = 3
    counter = 0
    
    for epoch in range(epochs):
        start_time = time.time()
        
        # Training phase
        model.train()
        train_loss = 0
        
        for batch_idx, (doc_ids, word_ids, context_ids) in enumerate(train_loader):
            # Chuyển dữ liệu lên device
            doc_ids = doc_ids.to(device)
            word_ids = word_ids.to(device)
            context_ids = context_ids.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(doc_ids, word_ids)
            loss = criterion(outputs, context_ids)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)  # Gradient clipping
            optimizer.step()
            
            train_loss += loss.item()
            
            # Giải phóng bộ nhớ
            del doc_ids, word_ids, context_ids, outputs
            if device.type == 'cuda':
                torch.cuda.empty_cache()
            
            # In tiến trình
            if batch_idx % 20 == 0:
                print(f'Epoch {epoch+1}/{epochs} [{batch_idx}/{len(train_loader)}] - Loss: {loss.item():.4f}')
        
        # Tính loss trung bình trên tập train
        avg_train_loss = train_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for doc_ids, word_ids, context_ids in val_loader:
                # Chuyển dữ liệu lên device
                doc_ids = doc_ids.to(device)
                word_ids = word_ids.to(device)
                context_ids = context_ids.to(device)
                
                # Forward pass
                outputs = model(doc_ids, word_ids)
                loss = criterion(outputs, context_ids)
                val_loss += loss.item()
                
                # Giải phóng bộ nhớ
                del doc_ids, word_ids, context_ids, outputs
                if device.type == 'cuda':
                    torch.cuda.empty_cache()
        
        # Tính loss trung bình trên tập validation
        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        # Điều chỉnh learning rate
        scheduler.step(avg_val_loss)
        
        # Lưu model tốt nhất
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_doc2vec_model.pth')
            print(f'Model được lưu với val_loss: {best_val_loss:.4f}')
            counter = 0  # Reset counter
        else:
            counter += 1
            
        # Early stopping
        if counter >= patience:
            print(f'Early stopping sau {epoch+1} epochs')
            break
            
        # In kết quả epoch
        time_elapsed = time.time() - start_time
        print(f'Epoch {epoch+1}/{epochs} kết thúc sau {time_elapsed:.1f}s - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
    
    # Vẽ đồ thị loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig('doc2vec_training_loss.png')
    plt.show()
    
    return train_losses, val_losses

In [ ]:
# Chuẩn bị DataLoader
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

# Khởi tạo model
doc_count = len(train_texts) + len(val_texts)
model = Doc2VecWithPretrained(
    pretrained_path=pretrained_path,
    vocab_size=text_dataset.vocab_size,
    doc_count=doc_count,
    vector_size=VECTOR_SIZE
)

# Gán từ điển cho model để khởi tạo embeddings
model.word2idx = text_dataset.word2idx

# Huấn luyện model
EPOCHS = 10
LEARNING_RATE = 0.001

train_losses, val_losses = train_doc2vec(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=EPOCHS,
    learning_rate=LEARNING_RATE
)

## 6. Tạo Document Embeddings

Sử dụng mô hình đã huấn luyện để tạo document embeddings cho các văn bản.

In [ ]:
def get_document_embeddings(model, texts, text_dataset):
    """
    Tạo document embeddings cho danh sách các văn bản
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    doc_embeddings = []
    
    with torch.no_grad():
        for text in texts:
            # Tokenize text
            word_indices = text_dataset.tokenize_text(text)
            if len(word_indices) == 0:  # Skip empty texts
                doc_embeddings.append(np.zeros(model.vector_size))
                continue
                
            # Convert to tensors
            word_indices = torch.tensor(word_indices, dtype=torch.long).to(device)
            
            # Get word embeddings
            word_embeds = model.word_embeddings(word_indices)
            
            # Average word embeddings to get document embedding
            doc_embedding = torch.mean(word_embeds, dim=0).cpu().numpy()
            doc_embeddings.append(doc_embedding)
    
    return np.array(doc_embeddings)

# Tải model tốt nhất
best_model = Doc2VecWithPretrained(
    pretrained_path=pretrained_path,
    vocab_size=text_dataset.vocab_size,
    doc_count=doc_count,
    vector_size=VECTOR_SIZE
)
best_model.word2idx = text_dataset.word2idx

try:
    best_model.load_state_dict(torch.load('best_doc2vec_model.pth'))
    print("Đã tải model tốt nhất từ file")
except:
    print("Không tìm thấy file model tốt nhất, sử dụng model hiện tại")
    best_model = model

# Tạo document embeddings
all_texts = df['text'].tolist()
all_labels = df['label'].tolist()
doc_embeddings = get_document_embeddings(best_model, all_texts, text_dataset)

print(f"Kích thước document embeddings: {doc_embeddings.shape}")

## 7. Trực quan hóa Document Embeddings

Trực quan hóa document embeddings sử dụng t-SNE để giảm chiều xuống 2D.

In [ ]:
def visualize_embeddings(embeddings, labels):
    """
    Trực quan hóa embeddings bằng t-SNE
    """
    # Sử dụng t-SNE để giảm chiều
    tsne = TSNE(n_components=2, random_state=42)
    reduced_embeddings = tsne.fit_transform(embeddings)
    
    # Tạo DataFrame để visualization
    vis_df = pd.DataFrame({
        'x': reduced_embeddings[:, 0],
        'y': reduced_embeddings[:, 1],
        'label': labels
    })
    
    # Các nhãn duy nhất
    unique_labels = vis_df['label'].unique()
    colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))
    
    # Visualize
    plt.figure(figsize=(12, 8))
    
    for i, label in enumerate(unique_labels):
        subset = vis_df[vis_df['label'] == label]
        plt.scatter(subset['x'], subset['y'], c=[colors[i]], label=label, alpha=0.7)
    
    plt.legend()
    plt.title('t-SNE Visualization of Document Embeddings')
    plt.savefig('doc2vec_visualization.png')
    plt.show()

# Trực quan hóa
visualize_embeddings(doc_embeddings, all_labels)

## 8. Ứng dụng: Tìm kiếm văn bản tương tự

Sử dụng document embeddings để tìm các văn bản tương tự với một văn bản mới.

In [ ]:
def compute_similarity(doc_embedding, all_embeddings):
    """
    Tính độ tương tự cosine giữa một document embedding với tất cả các document embeddings khác
    """
    # Normalize các vector
    doc_norm = np.linalg.norm(doc_embedding)
    all_norms = np.linalg.norm(all_embeddings, axis=1)
    
    # Tính cosine similarity
    similarities = np.dot(all_embeddings, doc_embedding) / (all_norms * doc_norm)
    return similarities

def find_similar_documents(query_text, model, text_dataset, all_texts, top_n=3):
    """
    Tìm top_n văn bản tương tự với query_text
    """
    # Tính document embedding cho query
    query_embedding = get_document_embeddings(model, [query_text], text_dataset)[0]
    
    # Tính document embeddings cho tất cả văn bản
    all_embeddings = get_document_embeddings(model, all_texts, text_dataset)
    
    # Tính độ tương tự
    similarities = compute_similarity(query_embedding, all_embeddings)
    
    # Lấy top_n văn bản tương tự nhất
    top_indices = np.argsort(similarities)[::-1][:top_n+1]  # +1 vì có thể bao gồm chính query
    
    # Trả về kết quả
    results = []
    for i in top_indices:
        if all_texts[i] != query_text:  # Bỏ qua nếu là chính query
            results.append({
                'text': all_texts[i],
                'similarity': similarities[i]
            })
    
    return results[:top_n]

# Ví dụ sử dụng
query = "Deep learning is changing how we build AI systems."
similar_docs = find_similar_documents(query, best_model, text_dataset, all_texts, top_n=3)

print(f"Query: {query}")
print("\nCác văn bản tương tự:")
for i, doc in enumerate(similar_docs):
    print(f"\n{i+1}. {doc['text']}")
    print(f"   Similarity: {doc['similarity']:.4f}")

## 9. Kết luận

Chúng ta đã thực hiện việc biểu diễn Doc2vec sử dụng Deep Learning với pretrained models:

1. **Pretrained Word Embeddings**: Sử dụng GloVe để khởi tạo word embeddings
2. **Kiến trúc Deep Learning**: Xây dựng mô hình với nhiều lớp ẩn (3 lớp), batch normalization và dropout
3. **Document Embeddings**: Tạo các vector biểu diễn cho văn bản bằng cách kết hợp word embeddings và doc embeddings

Ưu điểm của phương pháp này:
- Tận dụng được kiến thức từ pretrained word embeddings (transfer learning)
- Mô hình deep learning cho phép học các biểu diễn phức tạp hơn
- Document embeddings có thể sử dụng cho nhiều tác vụ NLP khác nhau

Đây là một cách tiếp cận hiệu quả để biểu diễn văn bản dưới dạng vector, kết hợp được cả pretrained knowledge và khả năng học của deep learning.